In [1]:
import pandas as pd

In [2]:
arch = pd.read_csv("./data/emapper/eggnog_non_viral_rhodo_top_orfs_emapper_arch.tsv", sep="\t",\
                   names=["orf", "seed_ortho", "em_evalue", "em_score", "name", "GO", "KEGG", "BiGG", "tax_scope", "eggNOGs", "best_OG","COG_cat","eggNOG_Desc"])

In [50]:
arch.shape

(487, 14)

In [53]:
arch.head()

,orf,seed_ortho,em_evalue,em_score,name,GO,KEGG,BiGG,tax_scope,eggNOGs,best_OG,COG_cat,eggNOG_Desc,kingdom
0,SAMEA2621155_1668496_7,246969.TAM4_1436,8.900000e-152,499.8,ACSA,"GO:0003674,GO:0003824,GO:0008150,GO:0008152,GO...","K01895,K14467,K15018",NaN,arNOG[11],"0KRI7@eurNOG,COG0365@NOG,arCOG01529@arNOG",arCOG01529|6.25061175138e-197|656.883361816,I,synthetase,arch
1,SAMEA2621155_2236281_3,246969.TAM4_1436,5.200000e-155,510.5,ACSA,"GO:0003674,GO:0003824,GO:0008150,GO:0008152,GO...","K01895,K14467,K15018",NaN,arNOG[11],"0KRI7@eurNOG,COG0365@NOG,arCOG01529@arNOG",arCOG01529|4.8768052731e-202|673.739746094,I,synthetase,arch
2,SAMEA2621155_98458_12,634498.mru_0164,4.200000e-93,305.4,SRP54,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",K03106,NaN,arNOG[11],"0KR9C@eurNOG,0WR8F@metNOG,COG0541@NOG,arCOG012...",arCOG01228|1.4727573295e-116|390.875610352,U,Involved in targeting and insertion of nascent...,arch
3,SAMEA2591098_31178_22,521011.Mpal_1279,3.000000e-80,260.7,NaN,NaN,NaN,NaN,arNOG[11],"COG2303@NOG,arCOG02232@arNOG",arCOG02232|1.64963181477e-32|103.813285828,E,glucose-methanol-choline oxidoreductase,arch
4,SAMEA2622568_61469_1,273075.Ta0421,4.600000e-119,391.6,ACSA,"GO:0003674,GO:0003824,GO:0008150,GO:0008152,GO...","K01895,K14467,K15018",NaN,arNOG[11],"0KRI7@eurNOG,1CCK9@theNOG,COG0365@NOG,arCOG015...",arCOG01529|1.24751588522e-155|520.46081543,I,synthetase,arch


In [3]:
bact = pd.read_csv("./data/emapper/eggnog_non_viral_rhodo_top_orfs_emapper_bact.tsv", sep="\t",\
                   names=["orf", "seed_ortho", "em_evalue", "em_score", "name", "GO", "KEGG", "BiGG", "tax_scope", "eggNOGs", "best_OG","COG_cat","eggNOG_Desc"])

In [51]:
bact.shape

(641, 15)

In [4]:
euk = pd.read_csv("./data/emapper/eggnog_non_viral_rhodo_top_orfs_emapper_euk.tsv", sep="\t",\
                   names=["orf", "seed_ortho", "em_evalue", "em_score", "name", "GO", "KEGG", "BiGG", "tax_scope", "eggNOGs", "best_OG","COG_cat","eggNOG_Desc"])

In [52]:
euk.shape

(525, 14)

In [5]:
arch["kingdom"] = "arch"
bact["kingdom"] = "bact"
euk["kingdom"] = "euk"

---

In [6]:
# The list was obtained from http://eggnogdb.embl.de/#/app/downloads on 2018/06/11
# ogs = pd.read_clipboard(names=["NOG_web"], usecols=[0])
# ogs["tax"] = ogs.NOG_web.apply(lambda x: x.split(" ")[0])
# ogs["NOG"] = ogs.NOG_web.apply(lambda x: x.split(" ")[1].strip("()"))
# ogs[["NOG", "tax"]].to_csv("./data/emapper/eggNOGs_tax.csv", index=False)

In [7]:
OGs = pd.read_csv("./data/emapper/eggNOGs_tax.csv", names=["nog_dataset", "nog_phylum"], skiprows=1)

In [8]:
OGs.head()

,nog_dataset,nog_phylum
0,aciNOG,Acidobacteria
1,acidNOG,Acidobacteriia
2,acoNOG,Aconoidasida
3,actNOG,Actinobacteria
4,agaNOG,Agaricales


In [9]:
pro_dict = {x:"proNOG" for x in "aproNOG;bproNOG;dproNOG;delNOG;gproNOG;proNOG".split(";")}

#pro_dict_rev = {"proNOG":x for x in "aproNOG;bproNOG;dproNOG;delNOG;gproNOG;proNOG".split(";")}

In [12]:
bactNOGs = set()
multiNOGs = set()
for i,df_ in bact.iterrows():
    best_og = df_["best_OG"].split("|")[0]
    nog_dataset_dict = {x.split("@")[0]:x.split("@")[1] for x in df_["eggNOGs"].split(",")}

    nog_dataset = nog_dataset_dict[best_og]
    if nog_dataset in ["bactNOG", "NOG"]:
        bactNOGs.add(i)
        nog_dataset_dict_rev = {x.split("@")[1]:x.split("@")[0] for x in df_["eggNOGs"].split(",")}
        
        nog_dataset_dict_rev.pop("NOG", None)
        nog_dataset_dict_rev.pop("bactNOG", None)
        
        pro_set = set()
        
        try:
            pro_set = set([pro_dict[key] for key in nog_dataset_dict_rev.keys()])
        except KeyError:
            pass
            # print("This NOG is not in the pro_dict {}".format(str(i)))
        
        if len(pro_set) == 0:
            nog_dataset = ";".join(nog_dataset_dict_rev.keys())
        elif len(pro_set) > 1:
            print("Check {}".format(i))
        else:
            nog_dataset = 'proNOG'
        
        
        if nog_dataset in pro_dict.keys():
            #print("{}\t{}".format(i, df_["eggNOGs"]))
            nog_dataset = pro_dict[nog_dataset]

        
        if len(nog_dataset_dict_rev.keys()) > 1:
            multiNOGs.add(i)
        elif len(nog_dataset_dict_rev.keys()) == 0:
            nog_dataset = "bactNOG"

    elif nog_dataset in pro_dict.keys():
        nog_dataset = pro_dict[nog_dataset]
        
    bact.loc[i, "nog_dataset"] = nog_dataset

---

In [15]:
eggnog = pd.concat([arch, bact, euk])

/Users/jfloresu/.virtualenvs/jlab/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [16]:
eggnog.shape

(1653, 15)

In [17]:
# remove duplicates based on the score
eggnog.sort_values(by=["orf", "em_score"], ascending=False, inplace=True)

In [18]:
# parse contig name from ORF
eggnog["contig"] = eggnog.orf.apply(lambda x : "_".join(x.split("_")[:2]))

In [19]:
eggnog.drop_duplicates("orf", inplace=True)

In [20]:
eggnog.shape

(663, 16)

In [21]:
egg = eggnog.groupby("contig")

In [22]:
for i, g in egg:
    if len(g.kingdom) > 1:
        print(i)

In [23]:
eggnog.drop_duplicates("contig").shape

(663, 16)

In [24]:
OG_dict = {'euNOG[57]':'Eukaryotes', 'fuNOG[21]':'Fungi', 'bactNOG[38]':'Bacteria', 'maNOG[6]':"Mammals", 'arNOG[11]':"Archaea",
       'virNOG[6]':"Viridiplantae", 'meNOG[21]':"Animals", 'opiNOG[43]':"Opisthokonts", 'apiNOG[5]':"Apicomplexa", 'artNOG[6]':"Arthropoda",
       'nemNOG[3]':"Nematodes", 'fiNOG[1]':"Fishes"}

In [25]:
eggnog["taxa"] = eggnog.tax_scope.map(OG_dict)

In [29]:

eggnog.tax_scope.value_counts()

bactNOG[38]    471
arNOG[11]       49
virNOG[6]       30
euNOG[57]       30
meNOG[21]       26
artNOG[6]       18
fuNOG[21]       18
apiNOG[5]       14
maNOG[6]         3
opiNOG[43]       2
fiNOG[1]         1
nemNOG[3]        1
Name: tax_scope, dtype: int64

In [30]:
eggnog.loc[eggnog.tax_scope == "bactNOG[38]", "nog_dataset"].value_counts()

proNOG              260
bctoNOG;flaNOG       57
actNOG               24
flaNOG               20
bactNOG              19
bctoNOG              13
cloNOG;firmNOG        8
bctoNOG;cytNOG        6
cyaNOG                6
plaNOG                6
cytNOG                6
bctoNOG;sphNOG        6
chloNOG               6
verNOG;verrNOG        5
chlNOG                5
bacteNOG;bctoNOG      4
bacNOG;firmNOG        3
chlaNOG               2
fusoNOG               2
chloNOG;chlorNOG      2
firmNOG;negNOG        2
aquNOG                1
verNOG                1
deiNOG                1
chloNOG;dehNOG        1
therNOG               1
aciNOG                1
sphNOG                1
verrNOG               1
tenNOG                1
Name: nog_dataset, dtype: int64

In [31]:
eggnog.loc[eggnog.tax_scope == "bactNOG[38]", "nog_dataset"].unique()

array(['bctoNOG;flaNOG', 'proNOG', 'cyaNOG', 'bactNOG', 'bctoNOG',
       'bacteNOG;bctoNOG', 'actNOG', 'cytNOG', 'chloNOG;chlorNOG',
       'flaNOG', 'verNOG;verrNOG', 'plaNOG', 'chloNOG', 'chlaNOG',
       'deiNOG', 'bctoNOG;sphNOG', 'cloNOG;firmNOG', 'tenNOG', 'sphNOG',
       'chlNOG', 'aquNOG', 'verNOG', 'firmNOG;negNOG', 'bacNOG;firmNOG',
       'bctoNOG;cytNOG', 'fusoNOG', 'aciNOG', 'chloNOG;dehNOG', 'verrNOG',
       'therNOG'], dtype=object)

In [32]:
len(eggnog.loc[eggnog.tax_scope == "bactNOG[38]", "nog_dataset"].unique())

30

In [33]:
# gram positive

monoderm = ['Actinobacteria',
 'Firmicutes',
 'Thermotogae',
 'Chloroflexi',
 'Tenericutes',
 'Haloplasmatales',
 'Deinococcus-Thermus', "Deinococcusthermus"]

In [34]:
# gram negative

diderm = ['Proteobacteria',
 'Aquificae',
 'Chlamydiae',
 'Bacteroidetes',
 'Chlorobi',
 'Cyanobacteria',
 'Fibrobacteres',
 'Verrucomicrobia',"Verrucomicrobiae",
 'Planctomycetes',
 'Spirochetes',
 'Acidobacteria',
 'Gemmatimonadetes',
 'Spirochaetes',
 'Balneolaeota',
 'Rhodothermaeota', "Flavobacteriia", "Cytophagia", "Sphingobacteriia",
         "Fusobacteria"]

In [35]:
Unclassified_Bact = ["Bacteria"]

In [36]:
Eukaryotes = ['Eukaryotes', 'Fungi', 'Mammals', 'Archaea', 'Viridiplantae',
       'Animals', 'Opisthokonts', 'Apicomplexa', 'Arthropoda',
       'Nematodes', 'Fishes']

In [37]:
Archaea = ["Archaea"]

In [38]:
len(monoderm)

8

In [39]:
len(diderm)

20

In [40]:
OGs.loc[OGs.nog_phylum.isin(diderm)].shape

(15, 2)

In [41]:
OGs.loc[OGs.nog_phylum.isin(monoderm)].shape

(7, 2)

In [42]:
eggnog = eggnog.merge(OGs, on="nog_dataset", how="left")

In [43]:
eggnog["plot_classification"] = ""

In [44]:
eggnog.loc[((eggnog.kingdom == "bact") & (eggnog.taxa.isin(Unclassified_Bact)), "plot_classification")] = "Unclassified_Bact"
eggnog.loc[((eggnog.kingdom == "bact") & (eggnog.nog_phylum.isin(monoderm)), "plot_classification")] = "monoderm"
eggnog.loc[((eggnog.kingdom == "bact") & (eggnog.nog_phylum.isin(diderm)), "plot_classification")] = "diderm"

eggnog.loc[((eggnog.kingdom == "euk") & (eggnog.taxa.isin(Eukaryotes)), "plot_classification")] = "Eukaryotes"
eggnog.loc[((eggnog.kingdom == "arch") & (eggnog.taxa.isin(Archaea)), "plot_classification")] = "Archaea"

In [45]:
eggnog.loc[eggnog.plot_classification == "", "nog_phylum"].unique()

array([], dtype=object)

In [46]:
eggnog.plot_classification.unique()

array(['Eukaryotes', 'Unclassified_Bact', 'diderm', 'Archaea', 'monoderm'],
      dtype=object)

In [47]:
eggnog.loc[eggnog.kingdom != "bact", "taxa"].unique()

array(['Eukaryotes', 'Fungi', 'Mammals', 'Archaea', 'Viridiplantae',
       'Animals', 'Opisthokonts', 'Apicomplexa', 'Arthropoda',
       'Nematodes', 'Fishes'], dtype=object)

In [48]:
eggnog.head()

,BiGG,COG_cat,GO,KEGG,best_OG,eggNOG_Desc,eggNOGs,em_evalue,em_score,kingdom,name,nog_dataset,orf,seed_ortho,tax_scope,contig,taxa,nog_phylum,plot_classification
0,NaN,L,"GO:0000018,GO:0000019,GO:0000075,GO:0000109,GO...",K10843,KOG1123|6.29003587737e-32|95.4249572754,Dna repair helicase,"COG1061@NOG,KOG1123@euNOG",3.100000e-25,82.5,euk,ERCC3,NaN,SAMEA2623919_2273661_1,5911.EAR85010,euNOG[57],SAMEA2623919_2273661,Eukaryotes,NaN,Eukaryotes
1,NaN,T,"GO:0000226,GO:0003674,GO:0003824,GO:0004672,GO...","K00889,K08286,K08857,K13712",0IP8V|4.36012232354e-41|117.290359497,whole genome shotgun sequence,"0IMJW@euNOG,0IP8V@euNOG,COG0515@NOG,COG4642@NO...",5.900000e-34,111.5,euk,PIP5KL1,NaN,SAMEA2623919_2256086_9,5911.EAR95086,euNOG[57],SAMEA2623919_2256086,Eukaryotes,NaN,Eukaryotes
2,NaN,L,"GO:0000002,GO:0000166,GO:0000228,GO:0000723,GO...",K15255,12Q40|1.00752482066e-54|181.190750122,PIF1 5'-to-3' DNA helicase homolog (S. cerevis...,"03JBD@ascNOG,0PGDJ@fuNOG,12Q40@opiNOG,COG0507@...",3.200000e-31,102.1,euk,FG05522.1,NaN,SAMEA2623919_1299782_8,4896.SPBC887.14c.1,fuNOG[21],SAMEA2623919_1299782,Fungi,NaN,Eukaryotes
3,ENO,G,"GO:0005575,GO:0005576,GO:0005622,GO:0005623,GO...",K01689,05C70|2.57585781835e-213|711.770263672,Catalyzes the reversible conversion of 2- phos...,"05C70@bactNOG,099YR@bctoNOG,0P7QI@flaNOG,COG01...",6.500000e-206,680.4,bact,ENO,bctoNOG;flaNOG,SAMEA2623919_1141576_1,411154.GFO_2809,bactNOG[38],SAMEA2623919_1141576,Bacteria,NaN,Unclassified_Bact
4,ADSS,F,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",K01939,05C91|2.29089421284e-186|623.047973633,Plays an important role in the de novo pathway...,"01Q8W@aproNOG,05C91@bactNOG,16PIX@proNOG,COG01...",5.100000e-172,568.9,bact,PURA,proNOG,SAMEA2623868_3012728_14,582899.Hden_2405,bactNOG[38],SAMEA2623868_3012728,Bacteria,Proteobacteria,diderm


In [49]:
eggnog.to_csv("./data/emapper/eggnog_non_viral_rhodo_contigs_annotation.tsv", index=False, sep="\t")